# Model Context Protocol (MCP) Labs - PART 1

## Overview
Comprehensive MCP integration labs with **real-world use cases**:

### Part 1 Contains:
- **Lab 1**: MCP Basics - Understanding MCP Architecture
- **Lab 2**: Filesystem MCP - Document & Resume Management
- **Lab 3**: Email MCP (Gmail) - Automated Email Workflows
- **Lab 4**: Calendar MCP (Google Calendar) - Meeting & Booking Scheduling

### Part 2 Contains:
- **Lab 5**: Discord MCP - Team Communication & Notifications
- **Lab 6**: GitHub MCP - Code Portfolio Review
- **Lab 7**: Custom MCP Server - Domain-Specific Operations
- **Lab 8**: Multi-MCP Orchestration - Complete Workflows

### Use Cases
1. **HR Recruitment System** - Continuing from previous labs
2. **AirBnb Property Management** - Guest communication, booking management

### References
- [LangChain MCP Docs](https://docs.langchain.com/oss/python/langchain/mcp)
- [MCP Protocol](https://modelcontextprotocol.io/introduction)
- [MCP Servers](https://mcpservers.org)

## Setup and Installation

In [ ]:
# Install Python packages
!pip install -q -U langchain-openai langgraph langchain-core python-dotenv
!pip install -q -U langchain-mcp-adapters mcp
!pip install -q -U google-auth google-auth-oauthlib google-auth-httplib2
!pip install -q -U google-api-python-client

# Install MCP servers (Node.js based - run in terminal)
# npm install -g @modelcontextprotocol/server-filesystem
# npm install -g @modelcontextprotocol/server-gmail
# npm install -g @modelcontextprotocol/server-google-calendar
# npm install -g @modelcontextprotocol/server-github
# npm install -g discord-mcp-server

In [ ]:
# Core imports
import os
import json
import asyncio
from dotenv import load_dotenv
from typing import TypedDict, Literal, Any, Dict, List
from datetime import datetime, timedelta
from pathlib import Path

# LangChain imports
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage
from langchain_core.tools import tool, BaseTool

# LangGraph imports
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

# MCP imports
try:
    from langchain_mcp_adapters.client import MultiServerMCPClient
    from mcp import ClientSession, StdioServerParameters
    MCP_AVAILABLE = True
except ImportError:
    MCP_AVAILABLE = False
    print("⚠️ MCP adapters not installed. Labs will use simulated tools.")

# Load environment variables
load_dotenv()

# Verify API keys
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("Please set OPENAI_API_KEY in your environment")

# Initialize LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0)

print("✓ Core setup complete!")
print(f"✓ MCP Available: {MCP_AVAILABLE}")

## Sample Data - HR & AirBnb Use Cases

In [ ]:
# ============================================================================
# HR RECRUITMENT DATA
# ============================================================================

SAMPLE_RESUME = """
PRIYA SHARMA
Senior Software Engineer
priya.sharma@email.com | +91-98765-43210 | Bengaluru, Karnataka
GitHub: github.com/priya-sharma

SUMMARY
6+ years in full-stack development, specializing in Python, React, and AWS.

EXPERIENCE
Senior Software Engineer | InfoTech Solutions, Bengaluru | 2021-Present
- Led 4-member team building microservices architecture
- Reduced API latency by 45% through optimization
- Tech: Python, FastAPI, React, PostgreSQL, AWS

SKILLS
Python, JavaScript, FastAPI, React, AWS, Docker, Kubernetes, PostgreSQL
"""

HR_DATABASE = {
    "candidates": {
        "CAN001": {
            "name": "Priya Sharma",
            "email": "priya.sharma@email.com",
            "phone": "+91-98765-43210",
            "position": "Senior Backend Engineer",
            "status": "screening",
            "github": "priya-sharma",
            "discord_id": "priya_dev#1234",
            "resume_path": "./data/resumes/priya_sharma.pdf"
        }
    },
    "interviewers": {
        "INT001": {
            "name": "Rahul Verma",
            "email": "rahul.verma@techcorp.in",
            "calendar_id": "rahul.verma@techcorp.in"
        }
    }
}

# ============================================================================
# AIRBNB PROPERTY MANAGEMENT DATA
# ============================================================================

AIRBNB_DATABASE = {
    "properties": {
        "PROP001": {
            "name": "Cozy Goa Beach Villa",
            "location": "Candolim, Goa",
            "capacity": 6,
            "price_per_night": 8500,
            "amenities": ["WiFi", "Pool", "Kitchen", "Beach Access"]
        }
    },
    "bookings": {
        "BOOK001": {
            "property_id": "PROP001",
            "guest_name": "Sarah Johnson",
            "guest_email": "sarah.j@email.com",
            "checkin": "2025-10-20",
            "checkout": "2025-10-25",
            "guests": 4,
            "status": "confirmed"
        }
    }
}

print(f"✓ HR Database: {len(HR_DATABASE['candidates'])} candidates")
print(f"✓ AirBnb Database: {len(AIRBNB_DATABASE['properties'])} properties")

---

# LAB 1: MCP Basics - Understanding Architecture

**Concepts**:
- MCP Client-Server Model
- Transport Types: stdio, HTTP, SSE
- Tool Discovery and Invocation
- MultiServerMCPClient usage

In [ ]:
print("""
╔══════════════════════════════════════════════════════════════╗
║           MODEL CONTEXT PROTOCOL (MCP) ARCHITECTURE          ║
╚══════════════════════════════════════════════════════════════╝

┌─────────────┐
│   LLM App   │  (Your LangChain/LangGraph Application)
└──────┬──────┘
       │
       │ Uses langchain-mcp-adapters
       ▼
┌─────────────────────┐
│ MultiServerMCPClient│  (Manages multiple MCP connections)
└──────┬──────────────┘
       │
       ├──────────────┬──────────────┬──────────────┐
       ▼              ▼              ▼              ▼
  ┌─────────┐   ┌──────────┐   ┌──────────┐   ┌──────────┐
  │Filesystem│   │  Gmail   │   │ Calendar │   │ Discord  │
  │MCP Server│   │MCP Server│   │MCP Server│   │MCP Server│
  └─────────┘   └──────────┘   └──────────┘   └──────────┘

TRANSPORT TYPES:
• stdio: Server runs as subprocess (local tools)
• HTTP: Server runs independently (remote services)
• SSE: Server-Sent Events (real-time streaming)

KEY BENEFITS:
✓ Standardized protocol for tool integration
✓ Multiple servers used simultaneously
✓ Clean separation between LLM app and services
✓ Easy to add/remove capabilities
""")

# Example MCP Configuration
mcp_config = {
    "mcpServers": {
        "filesystem": {
            "command": "npx",
            "args": ["-y", "@modelcontextprotocol/server-filesystem", "./data"],
            "transport": "stdio"
        },
        "gmail": {
            "command": "npx",
            "args": ["-y", "@modelcontextprotocol/server-gmail"],
            "transport": "stdio",
            "env": {"GMAIL_CREDENTIALS": "/path/to/credentials.json"}
        },
        "calendar": {
            "command": "npx",
            "args": ["-y", "@modelcontextprotocol/server-google-calendar"],
            "transport": "stdio"
        }
    }
}

print("\n📝 Example MCP Configuration:")
print(json.dumps(mcp_config, indent=2))
print("\n✓ Lab 1 Complete!")

---

# LAB 2: Filesystem MCP - Document Management

**Use Cases**:
- HR: Resume storage and retrieval
- AirBnb: Property documents, contracts

**MCP Server**: `@modelcontextprotocol/server-filesystem`

In [ ]:
# Filesystem MCP Tools (Simulated for demo)

@tool
def read_file(file_path: str) -> str:
    """
    Read file content from filesystem.
    
    Args:
        file_path: Path to file
    """
    if "priya_sharma" in file_path:
        return SAMPLE_RESUME
    elif "contract" in file_path:
        return "RENTAL AGREEMENT - Goa Beach Villa\nProperty: PROP001..."
    return f"File not found: {file_path}"

@tool
def write_file(file_path: str, content: str) -> str:
    """Write content to file."""
    return f"✓ File written: {file_path} ({len(content)} bytes)"

@tool
def list_files(directory: str) -> str:
    """List files in directory."""
    listings = {
        "./data/resumes": ["priya_sharma.pdf", "arjun_mehta.pdf"],
        "./data/properties": ["prop001_photos/", "prop002_photos/"],
        "./data/contracts": ["BOOK001_contract.pdf"]
    }
    return json.dumps({"directory": directory, "files": listings.get(directory, [])}, indent=2)

filesystem_tools = [read_file, write_file, list_files]

filesystem_agent = create_react_agent(
    llm,
    tools=filesystem_tools,
    state_modifier="You are a Document Management Assistant for HR and AirBnb operations."
)

print("✓ Filesystem MCP Agent created")

In [ ]:
# Test Lab 2
print("\n🧪 Lab 2: Filesystem MCP\n" + "="*80)

# Test 1: Read resume
print("\n--- Test 1: Read Candidate Resume ---")
result1 = filesystem_agent.invoke({
    "messages": [HumanMessage(content="Read Priya Sharma's resume from ./data/resumes/priya_sharma.pdf and summarize her key skills.")]
})

for msg in result1['messages']:
    if isinstance(msg, AIMessage) and msg.content and not msg.tool_calls:
        print(f"\n🤖 {msg.content}")
        break

# Test 2: List property files
print("\n--- Test 2: List Property Documents ---")
result2 = filesystem_agent.invoke({
    "messages": [HumanMessage(content="List all files in ./data/properties directory.")]
})

for msg in result2['messages']:
    if isinstance(msg, AIMessage) and msg.content and not msg.tool_calls:
        print(f"\n🤖 {msg.content}")
        break

print("\n✓ Lab 2 Complete!")

---

# LAB 3: Email MCP (Gmail) - Automated Workflows

**Use Cases**:
- HR: Interview invitations, offer letters
- AirBnb: Booking confirmations, check-in instructions

**MCP Server**: `@modelcontextprotocol/server-gmail`

In [ ]:
# Gmail MCP Tools

@tool
def send_email(to: str, subject: str, body: str, cc: str = "") -> str:
    """
    Send email via Gmail.
    
    Args:
        to: Recipient email
        subject: Email subject
        body: Email body
        cc: CC recipients
    """
    import hashlib
    msg_id = hashlib.md5(f"{to}{subject}".encode()).hexdigest()[:12]
    
    print(f"""
📧 EMAIL SENT
───────────────────────────────────────
To: {to}
Subject: {subject}
Message ID: {msg_id}
───────────────────────────────────────
{body[:200]}...
""")
    return f"✓ Email sent. ID: {msg_id}"

@tool
def search_emails(query: str, max_results: int = 10) -> str:
    """Search emails in Gmail."""
    results = []
    if "interview" in query.lower():
        results.append({
            "from": "priya.sharma@email.com",
            "subject": "Re: Interview Invitation",
            "snippet": "Thank you for the invitation...",
            "date": "2025-10-10"
        })
    return json.dumps(results, indent=2)

email_tools = [send_email, search_emails]

email_agent = create_react_agent(
    llm,
    tools=email_tools,
    state_modifier="You are an Email Communication Assistant for HR and AirBnb. Write professional, personalized emails."
)

print("✓ Email MCP Agent created")

In [ ]:
# Test Lab 3
print("\n🧪 Lab 3: Email MCP\n" + "="*80)

# Test 1: HR Interview Invitation
print("\n--- Test 1: Send Interview Invitation ---")
result1 = email_agent.invoke({
    "messages": [HumanMessage(content="""
Send interview invitation to priya.sharma@email.com for:
- Position: Senior Backend Engineer
- Date: October 15, 2025 at 10:00 AM IST
- Interviewers: Rahul Verma, Vikram Singh
- Duration: 90 minutes
- Meeting: https://meet.google.com/abc-defg-hij
""")]
})

# Test 2: AirBnb Booking Confirmation
print("\n--- Test 2: Send Booking Confirmation ---")
result2 = email_agent.invoke({
    "messages": [HumanMessage(content="""
Send booking confirmation to sarah.j@email.com for:
- Property: Cozy Goa Beach Villa
- Check-in: October 20, 2025 (2:00 PM)
- Check-out: October 25, 2025 (11:00 AM)
- Guests: 4
- Total: ₹42,500
Include WiFi password and house rules.
""")]
})

print("\n✓ Lab 3 Complete!")

---

# LAB 4: Calendar MCP (Google Calendar) - Scheduling

**Use Cases**:
- HR: Interview scheduling
- AirBnb: Booking calendar, maintenance

**MCP Server**: `@modelcontextprotocol/server-google-calendar`

In [ ]:
# Calendar MCP Tools

@tool
def create_calendar_event(
    summary: str,
    start_time: str,
    end_time: str,
    attendees: str,
    description: str = "",
    location: str = ""
) -> str:
    """
    Create Google Calendar event.
    
    Args:
        summary: Event title
        start_time: Start (ISO format)
        end_time: End (ISO format)
        attendees: Comma-separated emails
        description: Event description
        location: Location or meeting link
    """
    import hashlib
    event_id = hashlib.md5(f"{summary}{start_time}".encode()).hexdigest()[:12]
    
    print(f"""
📅 CALENDAR EVENT CREATED
───────────────────────────────────────
Event: {summary}
Start: {start_time}
End: {end_time}
Attendees: {attendees}
Location: {location}
Event ID: {event_id}
""")
    return f"✓ Event created: {event_id}"

@tool
def check_availability(calendar_id: str, start_date: str, end_date: str) -> str:
    """Check calendar availability."""
    availability = {
        "calendar": calendar_id,
        "available_slots": [
            {"date": "2025-10-15", "slots": [{"start": "10:00", "end": "11:30"}]},
            {"date": "2025-10-16", "slots": [{"start": "11:00", "end": "12:30"}]}
        ]
    }
    return json.dumps(availability, indent=2)

@tool
def block_calendar_dates(calendar_id: str, start_date: str, end_date: str, reason: str) -> str:
    """Block calendar dates."""
    return f"✓ Blocked {calendar_id} from {start_date} to {end_date}. Reason: {reason}"

calendar_tools = [create_calendar_event, check_availability, block_calendar_dates]

calendar_agent = create_react_agent(
    llm,
    tools=calendar_tools,
    state_modifier="You are a Calendar & Scheduling Assistant. Check availability before creating events."
)

print("✓ Calendar MCP Agent created")

In [ ]:
# Test Lab 4
print("\n🧪 Lab 4: Calendar MCP\n" + "="*80)

# Test 1: Schedule Interview
print("\n--- Test 1: Schedule Technical Interview ---")
result1 = calendar_agent.invoke({
    "messages": [HumanMessage(content="""
Schedule interview for Priya Sharma:
- Check Rahul Verma's availability on Oct 15, 2025
- Create 90-minute event at 10:00 AM IST
- Attendees: priya.sharma@email.com, rahul.verma@techcorp.in, vikram.singh@techcorp.in
- Title: Technical Interview - Priya Sharma
- Meeting: https://meet.google.com/abc-defg-hij
""")]
})

# Test 2: Block Property Calendar
print("\n--- Test 2: Block Property Calendar for Booking ---")
result2 = calendar_agent.invoke({
    "messages": [HumanMessage(content="""
Block Goa Beach Villa calendar:
- Property: prop001@calendar.com
- Dates: October 20-25, 2025
- Reason: Booking BOOK001 - Sarah Johnson
""")]
})

print("\n✓ Lab 4 Complete!")

---

## 🎯 Part 1 Summary

### Completed Labs:
✅ **Lab 1**: MCP Architecture - Understanding protocol and configuration

✅ **Lab 2**: Filesystem MCP - Document management for HR and AirBnb

✅ **Lab 3**: Email MCP - Interview invitations and booking confirmations

✅ **Lab 4**: Calendar MCP - Interview scheduling and booking calendar

### Next Steps:
Continue to **Part 2** for:
- Lab 5: Discord MCP
- Lab 6: GitHub MCP
- Lab 7: Custom MCP Servers
- Lab 8: Multi-MCP Orchestration

### Key Patterns Learned:
1. MCP server configuration and transport types
2. Using `create_react_agent` with MCP tools
3. Simulated vs real MCP server connections
4. Tool composition for domain-specific agents

All tools are currently **simulated** for learning. In production:
- Install actual MCP servers via npm
- Configure OAuth credentials
- Use `MultiServerMCPClient` for real connections